In [1]:
import os
def create_folder(folder_name):
    path = os.getcwd()
    n_f = path+"/splitted_dataset"+"/"+folder_name
    try:
        os.mkdir(n_f)
    except OSError:
        print ("Creation of the directory %s failed" % path)
    else:
        print ("Successfully created the directory %s " % path)
        return n_f
    return None

In [2]:
import numpy as np
def split(ds, labels: np.ndarray, perc):
    split_point = int(np.ceil(len(ds)*perc))
    test_dim = len(ds)-split_point
    ones = 0
    zeros = 0
    train_inserted = 0
    test_inserted = 0
    train_ds = np.zeros((split_point,32,16516))
    train_labels = np.zeros((split_point,), dtype=np.int64)
    test_ds = np.zeros((test_dim, 32, 16516))
    test_labels = np.zeros((test_dim,),  dtype=np.int64)
    for data, label in zip(ds,labels):
        if label == 0:
            if zeros < np.ceil(split_point/2) and train_inserted < split_point:
                train_ds[train_inserted] = data
                train_labels[train_inserted] = label
                zeros +=1
                train_inserted += 1
            else:
                test_ds[test_inserted] = data
                test_labels[test_inserted] = label
                test_inserted += 1
        if label == 1:
            if ones < np.ceil(split_point/2) and train_inserted < split_point:
                train_ds[train_inserted] = data
                train_labels[train_inserted] = label
                ones +=1
                train_inserted += 1
            else:
                test_ds[test_inserted] = data
                test_labels[test_inserted] = label
                test_inserted += 1

    return train_ds, train_labels, test_ds, test_labels

In [3]:
def unison_shuffled_copies(a, b):
    p = np.random.permutation(len(a))
    return a[p], b[p]

In [4]:
dataset = np.load("dataset_overlap.npy")
classes = np.load("/home/andrea/Scrivania/MIVIA_ROAD_DB1/bin_classes.npy")


In [5]:
shuffled_dataset, shuffled_labels = unison_shuffled_copies(dataset,classes)

In [6]:
train_ds, train_labels, test_ds, test_labels = split(shuffled_dataset, shuffled_labels, 0.75)

In [7]:
print((train_labels == 0).sum())
print((train_labels == 1).sum())
print((test_labels == 0).sum())
print((test_labels == 1).sum())
print(len(train_ds)==len(train_labels))
print(len(test_ds)==len(test_labels))

278
279
64
121
True
True


In [8]:
folders = ["fold1","fold2","fold3","fold4","fold5"]

In [10]:
def create_folds(num_folds, train_set, labels, test_set, test_labels):
    chunk_size = int(len(train_set)/num_folds)
    indexes = set(list(range(len(train_set))))
    for i in range(num_folds):
        folder = create_folder(folders[i])
        if folder is None:
            print("Errore")
            break
        if i < num_folds-1:
            val_ds_fold = train_set[chunk_size*i:chunk_size*(i+1)]
            val_labels_fold = labels[chunk_size*i:chunk_size*(i+1)]
            train_ds_fold = train_set[list(indexes.difference(list(range(chunk_size*i,chunk_size*(i+1)))))]
            train_labels_fold = labels[list(indexes.difference(list(range(chunk_size*i,chunk_size*(i+1)))))]
        else:
            val_ds_fold = train_set[chunk_size*i:]
            val_labels_fold = labels[chunk_size*i:]
            train_ds_fold = train_set[:chunk_size*i]
            train_labels_fold = labels[:chunk_size*i]
        np.save(folder+'/train_ds.npy',train_ds_fold.astype(np.int16))
        np.save(folder+'/train_labels.npy',train_labels_fold)
        np.save(folder+'/val_ds.npy',val_ds_fold.astype(np.int16))
        np.save(folder+'/val_labels.npy',val_labels_fold)
    path = os.getcwd()
    ds = path+"/splitted_dataset/"
    np.save(ds+"test_set.npy", test_set.astype(np.int16))
    np.save(ds+"test_labels.npy", test_labels)

In [11]:
create_folds(5, train_ds, train_labels, test_ds, test_labels)

Successfully created the directory /home/andrea/Scrivania/RC_Carta 
Successfully created the directory /home/andrea/Scrivania/RC_Carta 
Successfully created the directory /home/andrea/Scrivania/RC_Carta 
Successfully created the directory /home/andrea/Scrivania/RC_Carta 
Successfully created the directory /home/andrea/Scrivania/RC_Carta 
